In [5]:
qa_testing_system_prompt = """
You are an elite QA engineer and test automation expert, specialized in executing and evaluating test cases against backend code. Your task is to analyze test cases and backend code to determine which tests pass or fail.

### Objective
Given test cases in JSON format and corresponding backend code files, your goal is to:
- **Analyze** the test cases and their expected behavior
- **Review** the backend code implementation
- **Execute** the test cases virtually by analyzing the code
- **Determine** which tests pass or fail based on the implementation
- **Provide** detailed results in a structured JSON format

### Chain of Thought

1. **Understand**:
    - Review each test case's description and steps
    - Identify the expected behavior and outcomes
    - Analyze the backend code implementation

2. **Analyze**:
    - Map test cases to corresponding backend code
    - Identify the relevant functions and methods
    - Check if the implementation matches the expected behavior

3. **Evaluate**:
    - For each test case:
        - Verify if the implementation handles the test scenario correctly
        - Check if edge cases and error conditions are properly handled
        - Validate the expected outputs and behaviors

4. **Determine Status**:
    - Mark a test as "pass" if:
        - The implementation correctly handles all test steps
        - Expected outputs match the actual behavior
        - Error conditions are properly managed
    - Mark a test as "fail" if:
        - The implementation doesn't match expected behavior
        - Edge cases are not handled correctly
        - Error conditions are not properly managed
        - Expected outputs don't match actual behavior

5. **Final Answer**:
    - Output test results in a JSON format, using the following structure:
    ```json
    {
      "test_results": [
        {
          "test_id": "unique_test_id_001",
          "description": "Brief description of what the test case verifies",
          "status": "pass|fail",
          "actual_result": "Detailed explanation of what actually happened",
          "expected_result": "What was expected to happen",
          "failure_reason": "If failed, explain why (null if passed)"
        }
      ],
      "summary": {
        "total_tests": 0,
        "passed": 0,
        "failed": 0,
        "pass_percentage": 0
      }
    }
    ```

### Guidelines:
- **Be thorough** in analyzing both test cases and code
- **Provide detailed explanations** for both passing and failing tests
- **Consider all edge cases** and error conditions
- **Be precise** in matching test expectations with actual implementation
- **Document any assumptions** made during the analysis

### What Not to Do:
- **Never** mark a test as passed without proper verification
- **Do not** ignore edge cases or error conditions
- **Never** provide vague or unclear explanations
- **Avoid** making assumptions about functionality not clearly implemented
- **Do not** skip analyzing any part of the test case or code
""" 

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

json_output_parser = JsonOutputParser()

json_prompt_template = PromptTemplate(
    template="{system_prompt} \n {format_instruction} \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
    partial_variables={"format_instruction" : json_output_parser.get_format_instructions()}
)

prompt_template = PromptTemplate(
    template="{system_prompt} \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
)



In [7]:
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
load_dotenv()
import os 

# os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGSMITH_API_KEY")
os.environ['LANGSMITH_PROJECT'] = os.getenv("LANGSMITH_PROJECT")
os.environ['LANGSMITH_TRACING'] = os.getenv("LANGSMITH_TRACING")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

from langchain_groq import ChatGroq 
from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
# llm = ChatGroq(model="qwen-2.5-32b")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [8]:
def perform_qa_testing_with_llm(test_cases, backend_code):
    user_query =  f"Perform qa testing for the test cases {test_cases} for the this backend code: {backend_code}"
    chain = json_prompt_template | llm | json_output_parser
    response = chain.invoke({"system_prompt" : qa_testing_system_prompt, "human_query" : user_query})
    return response

In [9]:
backend_code = """
Let's create a Node.js backend for the financial app with proper structure and security features.

<boltArtifact id="financial-app-backend" title="Financial App Backend Setup">
<boltAction type="file" filePath="package.json">
{
  "name": "financial-app-backend",
  "version": "1.0.0",
  "private": true,
  "type": "module",
  "scripts": {
    "start": "node src/server.js",
    "dev": "nodemon src/server.js"
  },
  "dependencies": {
    "express": "^4.18.2",
    "express-validator": "^7.0.1",
    "jsonwebtoken": "^9.0.2",
    "bcryptjs": "^2.4.3",
    "cors": "^2.8.5",
    "dotenv": "^16.3.1"
  },
  "devDependencies": {
    "nodemon": "^3.0.2"
  }
}
</boltAction>

<boltAction type="file" filePath="src/server.js">
import express from 'express';
import cors from 'cors';
import dotenv from 'dotenv';
import authRoutes from './routes/auth.js';
import bankRoutes from './routes/bank.js';
import loanRoutes from './routes/loan.js';
import billRoutes from './routes/bill.js';
import insuranceRoutes from './routes/insurance.js';

dotenv.config();

const app = express();
const port = process.env.PORT || 3000;

app.use(cors());
app.use(express.json());

app.use('/api/auth', authRoutes);
app.use('/api/bank', bankRoutes);
app.use('/api/loan', loanRoutes);
app.use('/api/bill', billRoutes);
app.use('/api/insurance', insuranceRoutes);

app.listen(port, () => console.log(`Server running on port ${port}`));
</boltAction>

<boltAction type="file" filePath="src/routes/auth.js">
import express from 'express';
import { body, validationResult } from 'express-validator';
import jwt from 'jsonwebtoken';
import bcrypt from 'bcryptjs';

const router = express.Router();

router.post('/mfa/enable', 
  body('userId').notEmpty(),
  body('mfaType').isIn(['biometric', 'mpin']),
  (req, res) => {
    const errors = validationResult(req);
    if (!errors.isEmpty()) {
      return res.status(400).json({ errors: errors.array() });
    }
    res.json({ message: 'MFA enabled successfully' });
});

router.post('/mfa/verify', 
  body('userId').notEmpty(),
  body('mfaToken').notEmpty(),
  (req, res) => {
    res.json({ message: 'MFA verification successful' });
});

export default router;
</boltAction>

<boltAction type="file" filePath="src/routes/bank.js">
import express from 'express';
import { body } from 'express-validator';

const router = express.Router();

router.post('/link', 
  body('accountNumber').notEmpty(),
  body('bankName').notEmpty(),
  (req, res) => {
    res.json({ message: 'Bank account linked successfully' });
});

router.get('/accounts', (req, res) => {
  res.json({ accounts: [] });
});

router.put('/default/:accountId', (req, res) => {
  res.json({ message: 'Default account updated' });
});

export default router;
</boltAction>

<boltAction type="file" filePath="src/routes/loan.js">
import express from 'express';
import { body } from 'express-validator';

const router = express.Router();

router.post('/apply',
  body('amount').isNumeric(),
  body('term').isNumeric(),
  (req, res) => {
    res.json({
      loanId: '123',
      status: 'PENDING',
      amount: req.body.amount,
      interest: '12%'
    });
});

router.get('/status/:loanId', (req, res) => {
  res.json({ status: 'APPROVED' });
});

export default router;
</boltAction>

<boltAction type="file" filePath="src/routes/bill.js">
import express from 'express';
import { body } from 'express-validator';

const router = express.Router();

router.post('/add-biller',
  body('billerName').notEmpty(),
  body('billerId').notEmpty(),
  (req, res) => {
    res.json({ message: 'Biller added successfully' });
});

router.get('/pending', (req, res) => {
  res.json({ bills: [] });
});

router.post('/pay',
  body('billId').notEmpty(),
  body('amount').isNumeric(),
  (req, res) => {
    res.json({ transactionId: '123', status: 'SUCCESS' });
});

export default router;
</boltAction>

<boltAction type="file" filePath="src/routes/insurance.js">
import express from 'express';
import { body } from 'express-validator';

const router = express.Router();

router.get('/policies', (req, res) => {
  res.json({ policies: [] });
});

router.post('/purchase',
  body('policyId').notEmpty(),
  body('term').isNumeric(),
  (req, res) => {
    res.json({ 
      policyNumber: '123',
      status: 'ACTIVE',
      coverage: '1000000'
    });
});

export default router;
</boltAction>
<boltAction type="shell">npm install && npm run dev</boltAction>
</boltArtifact>
"""

In [10]:
test_cases = [
  {
    "test_id": "TC001",
    "description": "Enable MFA with valid userId and mfaType",
    "steps": [
      "Send POST request to /api/auth/mfa/enable",
      "Include JSON body with userId: 'user123' and mfaType: 'mpin'",
      "Expect response status 200 and message 'MFA enabled successfully'"
    ],
    "status": "draft"
  },
  {
    "test_id": "TC002",
    "description": "Apply for a loan with valid amount and term",
    "steps": [
      "Send POST request to /api/loan/apply",
      "Include JSON body with amount: 50000 and term: 12",
      "Expect response with loanId, status: 'PENDING', and interest value"
    ],
    "status": "draft"
  },
  {
    "test_id": "TC003",
    "description": "Link a bank account with valid details",
    "steps": [
      "Send POST request to /api/bank/link",
      "Include JSON body with accountNumber: '1234567890' and bankName: 'BankX'",
      "Expect response status 200 and message 'Bank account linked successfully'"
    ],
    "status": "draft"
  },
  {
    "test_id": "TC004",
    "description": "Pay a bill with valid billId and amount",
    "steps": [
      "Send POST request to /api/bill/pay",
      "Include JSON body with billId: 'bill123' and amount: 1500",
      "Expect response with transactionId and status: 'SUCCESS'"
    ],
    "status": "draft"
  },
  {
    "test_id": "TC005",
    "description": "Purchase an insurance policy with valid policyId and term",
    "steps": [
      "Send POST request to /api/insurance/purchase",
      "Include JSON body with policyId: 'pol789' and term: 5",
      "Expect response with policyNumber, status: 'ACTIVE', and coverage amount"
    ],
    "status": "draft"
  }
]


In [11]:
perform_qa_testing_with_llm(test_cases, backend_code)

{'test_results': [{'test_id': 'TC001',
   'description': 'Enable MFA with valid userId and mfaType',
   'status': 'pass',
   'actual_result': "The request to /api/auth/mfa/enable with userId and mfaType in the body returns a 200 status code and the message 'MFA enabled successfully'. Validation is performed on the userId and mfaType.",
   'expected_result': "Response status 200 and message 'MFA enabled successfully'",
   'failure_reason': None},
  {'test_id': 'TC002',
   'description': 'Apply for a loan with valid amount and term',
   'status': 'pass',
   'actual_result': "The request to /api/loan/apply with amount and term in the body returns a loanId, status 'PENDING', and interest value. Validation is performed on the amount and term.",
   'expected_result': "Response with loanId, status: 'PENDING', and interest value",
   'failure_reason': None},
  {'test_id': 'TC003',
   'description': 'Link a bank account with valid details',
   'status': 'pass',
   'actual_result': "The request 

In [ ]:
{'test_results': [{'test_id': 'TC001',
   'description': 'Enable MFA with valid userId and mfaType',
   'status': 'pass',
   'actual_result': "The request to /api/auth/mfa/enable with userId and mfaType in the body returns a 200 status code and the message 'MFA enabled successfully'. Validation is performed on the userId and mfaType.",
   'expected_result': "Response status 200 and message 'MFA enabled successfully'",
   'failure_reason': None},
  {'test_id': 'TC002',
   'description': 'Apply for a loan with valid amount and term',
   'status': 'pass',
   'actual_result': "The request to /api/loan/apply with amount and term in the body returns a loanId, status 'PENDING', and interest value. Validation is performed on the amount and term.",
   'expected_result': "Response with loanId, status: 'PENDING', and interest value",
   'failure_reason': None},
  {'test_id': 'TC003',
   'description': 'Link a bank account with valid details',
   'status': 'pass',
   'actual_result': "The request to /api/bank/link with accountNumber and bankName in the body returns a 200 status code and the message 'Bank account linked successfully'. Validation is performed on the accountNumber and bankName.",
   'expected_result': "Response status 200 and message 'Bank account linked successfully'",
   'failure_reason': None},
  {'test_id': 'TC004',
   'description': 'Pay a bill with valid billId and amount',
   'status': 'pass',
   'actual_result': "The request to /api/bill/pay with billId and amount in the body returns a transactionId and status 'SUCCESS'. Validation is performed on the billId and amount.",
   'expected_result': "Response with transactionId and status: 'SUCCESS'",
   'failure_reason': None},
  {'test_id': 'TC005',
   'description': 'Purchase an insurance policy with valid policyId and term',
   'status': 'pass',
   'actual_result': "The request to /api/insurance/purchase with policyId and term in the body returns a policyNumber, status 'ACTIVE', and coverage amount. Validation is performed on the policyId and term.",
   'expected_result': "Response with policyNumber, status: 'ACTIVE', and coverage amount",
   'failure_reason': None}],
 
 
 'summary': {'total_tests': 5,
  'passed': 5,
  'failed': 0,
  'pass_percentage': 100}}